<a href="https://colab.research.google.com/github/dikshabhati1/song-lyrics-generator/blob/main/scraping_lyrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
client_secret = 'ynB0ibNICGS-hOgA-sBBj1rhVs9wgMi6wJjaytjmqDypzlJgbHmx6UCSmHcyZFoAhGr9YSkaZiwqnDNZEtrkwA'
client_id = '6rd56WOg4oPDiJppuGmgYoMM8XVK0bI1PQpSAqfpGOn5aIPm4QxoGmQ799RGy0op'
client_access_token = 'kOqWu7Mi_0pfKWXvbU9pUe3Jyt83znMmLxLXi_Szb8CN_L2NqkW2Wnc3oEy2t4Uh'

In [ ]:
!pip install lyricsgenius

In [4]:
import requests
from bs4 import BeautifulSoup
import os
import re
import pandas as pd
import lyricsgenius as lg 

First of all we are going to scrap the artist name

In [5]:
url = 'https://www.listchallenges.com/100-famous-male-singers'
page = requests.get(url)
soup = BeautifulSoup(page.text,'html.parser')

In [6]:
artist_name = []
for x in soup.findAll('div',class_ = 'item-name'):
    name = x.get_text()
    artist_name.append(name)

In [7]:
artist_name[:2]

['\r\n\t\t\t\t\t\t\t\t\t\t50 Cent\r\n\t\t\t\t\t\t\t\t\t',
 '\r\n\t\t\t\t\t\t\t\t\t\tAdam Lambert\r\n\t\t\t\t\t\t\t\t\t']

In [8]:
artist_name = [x.replace("\r\n",'') for x in artist_name]
artist_name = [x.replace("\t",'') for x in artist_name]

In [9]:
artist_name[:2]

['50 Cent', 'Adam Lambert']

In [10]:
len(artist_name)

40

Now we have scraped 40 artist name.It's time to scrap the lyrics

Now we are going to scrap the songs lyrics.We are going to take the name of artists in array and number of songs we want by each artist and then sort the songs by it's popularity so that each artist’s most popular songs are grabbed first.Then saves the song lyrics, song name and artist name in  the dataframe.We will find lyrics of our 40 artists songs and 25 songs for each artist

In [11]:
# Client access token from Genius Client API page
genius = lg.Genius(client_access_token,  
                             skip_non_songs=True, excluded_terms=["(Remix)", "(Live)"],
                             remove_section_headers=True)


# write lyrics of k songs by each artist in arr
 
lyrics_list = []
song_name = []
c = 0  
for name in artist_name:
    try:
        songs = (genius.search_artist(name,max_songs=25, sort='popularity')).songs
        s = [song.lyrics for song in songs]
        for i in s:
            lyrics_list.append(i)
        for j in songs:
            song_name.append(j)
        
        c += 1 
        print(f"Songs grabbed:{len(s)}")
    except:  #  Broad catch which will give us the name of artist and song that threw the exception
        print(f"Some exception at {name}: {c}")
df_ = pd.DataFrame()
df_['lyrics'] = lyrics_list
df_['song_name'] = song_name

Searching for songs by 50 Cent...

Some exception at 50 Cent: 0
Searching for songs by Adam Lambert...

Song 1: "Ghost Town"
Song 2: "Whataya Want From Me"
Song 3: "Runnin’"
Song 4: "Two Fux"
Song 5: "Another Lonely Night"
Song 6: "New Eyes"
Song 7: "For Your Entertainment"
Song 8: "If I Had You"
Song 9: "Superpower"
Song 10: "Underground"
Song 11: "There I Said It"
Song 12: "Welcome to the Show"
Song 13: "Closer to You"
Song 14: "Overglow"
Song 15: "Outlaws of Love"
Song 16: "Evil in the Night"
Song 17: "Better Than I Know Myself"
Song 18: "Stranger You Are"
Song 19: "The Original High"
Song 20: "Never Close Our Eyes"
Song 21: "Rumors"
Song 22: "Trespassing"
Song 23: "Comin In Hot"
Song 24: "Lucy"
Song 25: "Loverboy"

Reached user-specified song limit (25).
Done. Found 25 songs.
Songs grabbed:25
Searching for songs by Adam Levine...

"Girls Like You (Remix)" is not valid. Skipping.
Song 1: "Lost Stars"
"Locked Away Again (The Remix)" is not valid. Skipping.
"Wait (Remix)" is not valid

Let's look at our dataset

In [43]:
df_.head()

,lyrics,song_name
0,Died last night in my dreams\nWalking the stre...,"""Ghost Town"" by Adam Lambert:\n Died last n..."
1,"Hey, slow it down\nWhataya want from me?\nWhat...","""Whataya Want From Me"" by Adam Lambert:\n H..."
2,Steel to my trembling lips\nHow did the night ...,"""Runnin’"" by Adam Lambert:\n Steel to my tr..."
3,I only trust my tarot cards\nPink flamingos in...,"""Two Fux"" by Adam Lambert:\n I only trust m..."
4,Alone in the dark\nHole in my heart\nTurn on t...,"""Another Lonely Night"" by Adam Lambert:\n A..."


In [44]:
df_.shape

(683, 2)

In [45]:
# remove \n from lyrics
def lyric_clean(text):
    text = str(text).replace('\n','.') 
    return text
df_['lyrics'] = df_['lyrics'].apply(lambda x: lyric_clean(x))

In [46]:
def name_clean(text):
    text = re.findall(r'"([^"]*)"',text) 
    text = str(text).replace('[','') 
    text = str(text).replace(']','') 
    text = str(text).replace("'",'') 
    return text
df_['song_name'] = df_['song_name'].apply(lambda x: name_clean(x))

In [47]:
df_

,lyrics,song_name
0,Died last night in my dreams.Walking the stree...,Ghost Town
1,"Hey, slow it down.Whataya want from me?.Whatay...",Whataya Want From Me
2,Steel to my trembling lips.How did the night e...,Runnin’
3,I only trust my tarot cards.Pink flamingos in ...,Two Fux
4,Alone in the dark.Hole in my heart.Turn on the...,Another Lonely Night
...,...,...
678,"Just a little rush, babe, to feel dizzy.To der...",Sedated
679,She moved with shameless wonder.The perfect cr...,Foreigner’s God
680,I'd be the voice that urged Orpheus.When her b...,Talk
681,My head was warm.My skin was soaked.I called y...,In the Woods Somewhere


In [48]:
df_.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683 entries, 0 to 682
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   lyrics     683 non-null    object
 1   song_name  683 non-null    object
dtypes: object(2)
memory usage: 10.8+ KB


In [50]:
df_.to_csv('song-lyrics.csv',index=False)